In [1]:
#get records and check matches
import pymongo
import json
#from collections import Iterable
import pandas as pd
import spacy

In [2]:
connection = pymongo.MongoClient('localhost', 27017)

db = connection['my_phd']

collection = db['clustered_collections']

In [3]:
#get the devices, retrieve the information about the device
def get_device_attributes():
    batch = collection.find({"Anchor": "Device"})
    
    output = []
    for item in batch:
        output.append([item["Device_id"], item["type"], item["attribute"], item["Description"]])
    return(output)

In [3]:
nlp = spacy.load("en_core_web_lg")

#input 
keywords = ["temperature", "humidity", "measurment"]
description = "some device that is able to measure the temperature"


#function to collect synonyms
def get_synonyms(word):
    
    synonyms = []

    for syn in wordnet.synsets(word):
        for name in syn.lemma_names():
            synonyms.append(name)

    synonyms = list(dict.fromkeys(synonyms))
    synonyms = [x for x in synonyms if "_" not in x]
    
    synonyms_filtered = []
    
    for syn in synonyms:
        nlp_item_input = nlp(word)
        nlp_item_synonym = nlp(syn)
        similarity_coeff = nlp_item_input.similarity(nlp_item_synonym)
        if similarity_coeff > 0.4:
            synonyms_filtered.append(syn)
        
    return synonyms_filtered


#calculate the matching coefficient between the available items and the input parameters 
def calculate_matching_coefficient():
    device_info = get_device_attributes()

    result = []
    for device in device_info:
        device_description = device[3]
        device_keywords = device[2]
        
        #tokenize the descriptions
        doc1 = nlp(device_description)
        doc2 = nlp(description)
        
        #tokenize the keywords
        doc3 = nlp(' '.join(device_keywords))
        doc4 = nlp(' '.join(keywords))
        
        #calculate the coefficient to calculate the searching matches
        matches = []
        for search_item in doc4:
            search_item_lemma = search_item.lemma_
            
            for dev_item in doc3:
                dev_item_lemma = dev_item.lemma_
                synonyms = get_synonyms(search_item_lemma)
                
                if search_item_lemma == dev_item_lemma or search_item_lemma in synonyms:
                    matches.append(search_item_lemma)
        
        #calculating the number of keywords matching in a keyword's base form
        keywords_matches = len(matches) / len(doc4)
        
        #calculating the similarity using the language models of spacy library
        description_similarity = doc1.similarity(doc2)
        
        #calculating the cummulative coefficient
        cumul_coeff = (keywords_matches + description_similarity) / 2
        
        #get the result
        result.append([device[0], cumul_coeff])  
    
    return result

def select_best_option():
    matches = calculate_matching_coefficient()
    best_option = max(map(lambda x: x[1], matches))
    
    for match in matches:
        if match[1] == best_option:
            result = match[0]
            
            output = collection.find({"Device_id": result})
            
            for item in output:
                return item


select_best_option()

C:\Users\Artem\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'get_device_attributes' is not defined